Testes praticos utilizando o comparativo SKImage

In [2]:
import cv2
from skimage.metrics import structural_similarity as compare_ssim
import numpy as np

Função para redimencionar as duas imagens para que tenham o mesmo tamanho teste 1

In [3]:
# Função para redimensionar a segunda imagem para o tamanho da primeira
def resize_second_image_to_match(first_image_path, second_image_path):
    # Carregar a primeira imagem
    first_image = cv2.imread(first_image_path)
    
    # Carregar a segunda imagem
    second_image = cv2.imread(second_image_path)
    
    # Obter as dimensões da primeira imagem
    height, width = first_image.shape[:2]
    
    # Redimensionar a segunda imagem para corresponder ao tamanho da primeira
    resized_second_image = cv2.resize(second_image, (width, height))
    
    return first_image, resized_second_image


In [4]:
# Carregar as duas imagens
# imageA = cv2.imread('D:\Pos_IA\TCC\carroa.jpg')
# imageB = cv2.imread('D:\Pos_IA\TCC\carrob.jpg')

# Caminhos para as imagens
first_image_path = 'D:\Pos_IA\TCC\carroa.jpg'
second_image_path = 'D:\Pos_IA\TCC\carrob.jpg'

# Redimensionar a segunda imagem para corresponder à primeira
imageA, imageB = resize_second_image_to_match(first_image_path, second_image_path)

# Salvar a segunda imagem redimensionada, se necessário
cv2.imwrite('segunda_imagem_redimensionada.png', imageB)

True

In [5]:
# Converter as imagens para escala de cinza
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

In [6]:
# Calcular a semelhança estrutural entre as duas imagens
(score, diff) = compare_ssim(grayA, grayB, full=True)
print(f"Similaridade Estrutural: {score:.2%}")

Similaridade Estrutural: 29.83%


In [7]:
## não deu certo
# Supondo que grayA e grayB são suas imagens em escala de cinza 
(score, diff) = compare_ssim(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# Aplicar threshold para binarizar a imagem de diferença
_, thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

# Aplicar operações morfológicas para remover ruído e destacar as áreas de interesse
kernel = np.ones((3,3), np.uint8)
dilated = cv2.dilate(thresh, kernel, iterations=1)

# Encontrar contornos das áreas diferenciadas
contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar contornos nas imagens para demarcar as diferenças
for contour in contours:
    if cv2.contourArea(contour) > 40: # Filtrar pequenos contornos para reduzir ruído
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(imageA, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.rectangle(imageB, (x, y), (x+w, y+h), (0, 0, 255), 2)

# Exibir ou salvar as imagens com as diferenças demarcadas
cv2.imwrite('imageA_differences.png', imageA)
cv2.imwrite('imageB_differences.png', imageB)
# cv2.imshow('imageB_differences', imageA)

True

In [8]:
# Converte as imagens em escala cinza
gray_img1 = grayA 
gray_img2 = grayB 

# Calcula o histograma das imagens
histogram1 = cv2.calcHist([gray_img1],[0],None,[256],[0,256])
histogram2 = cv2.calcHist([gray_img2],[0],None,[256],[0,256])

# Encontra as diferenças entre os histograms
differences = np.abs(np.array(histogram1)-np.array(histogram2))

# Identifica os pontos mais diferentes
max_diff = differences.argmax()

# Encontra a posição do ponto de maior diferença
max_diff_position = (max_diff, int(histogram1[max_diff]))

# Desenha um círculo vermelho no ponto de maior diferença na primeira imagem
output_img = imageA.copy()
cv2.circle(output_img, (max_diff_position[0], 0), 5, (0, 0, 255), -1)

# Mostra a imagem com o ponto de diferença destacado
cv2.imshow('Imagem com Ponto de Diferença', output_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\Mau77\AppData\Local\Temp\ipykernel_14584\1189775290.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  max_diff_position = (max_diff, int(histogram1[max_diff]))


In [ ]:
# Converte as imagens em escala cinza
gray1 = grayA 
gray2 = grayB 

# Garantir que as imagens sejam do mesmo tamanho
gray1 = cv2.resize(gray1, (imageB.shape[1], imageB.shape[0]))

# Calcular a diferença absoluta entre as imagens
difference = cv2.absdiff(gray1, gray2)

# Limiarização para binarizar a imagem de diferença
_, thresh = cv2.threshold(difference, 30, 255, cv2.THRESH_BINARY)

# Encontrar contornos
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar contornos na imagem original
for contour in contours:
    if cv2.contourArea(contour) > 100:  # Filtrar pequenas diferenças
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(imageA, (x, y), (x + w, y + h), (0, 0, 255), 2)

# Salvar ou exibir a imagem resultante
cv2.imwrite('/mnt/data/difference_image.png', imageA)
cv2.imshow('Differences', imageA)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# A diferença é uma imagem flutuante, precisamos normalizar para 8-bit para visualizar
diff = (diff * 255).astype("uint8")

# Alinhando as imagens lado a lado
# Redimensionar imagens para o mesmo tamanho para comparação
height = max(imageA.shape[0], imageB.shape[0])
width = imageA.shape[1] + imageB.shape[1]
combined_image = np.zeros((height, width, 3), dtype=np.uint8)

In [ ]:
# Alinhar as imagens lado a lado para comparação
combined_image = np.hstack((imageA, imageB))

# Salvar a imagem combinada
cv2.imwrite('comparacao_lado_a_lado.png', combined_image)

# Mostrar a imagem combinada
cv2.imshow('Comparacao Lado a Lado', combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Função para redimensionar a segunda imagem para o tamanho da primeira
def resize_image_to_match(reference_image, target_image):
    # Obter as dimensões da imagem de referência
    height, width = reference_image.shape[:2]
    # Redimensionar a imagem alvo para corresponder ao tamanho da imagem de referência
    resized_image = cv2.resize(target_image, (width, height))
    return resized_image

In [ ]:
#Função para realizar o gradiente das duas imagens
def gradiente_horizontal(first_image_path, second_image_path):
    # Carregar as imagens
    first_image = cv2.imread(first_image_path)
    second_image = cv2.imread(second_image_path)
    
    # Redimensionar a segunda imagem para corresponder ao tamanho da primeira
    second_image_resized = resize_image_to_match(first_image, second_image)
    
    # Assegurar que as imagens são do tipo float para combinação
    first_image_float = first_image.astype(np.float32)
    second_image_float = second_image_resized.astype(np.float32)

    # Criar um degradê linear para a transparência
    height, width = first_image.shape[:2]
    gradient = np.linspace(0.3, 1, width)
    gradient = np.tile(gradient, (height, 1))
    gradient = np.dstack((gradient, gradient, gradient)).astype(np.float32)

    # Aplicar o degradê nas duas imagens
    first_image_weighted = cv2.multiply(first_image_float, gradient)
    second_image_weighted = cv2.multiply(second_image_float, 1 - gradient)

    # Combinar as imagens ponderadas
    combined_image_with_gradient = cv2.add(first_image_weighted, second_image_weighted)

    # Converter o resultado para uint8 para poder salvar e exibir
    combined_image_with_gradient = combined_image_with_gradient.astype(np.uint8)

    return combined_image_with_gradient

In [ ]:
# Carregar a primeira imagem e redimensionar a segunda para corresponder
first_image = cv2.imread(first_image_path)
second_image = cv2.imread(second_image_path)

# Chamando a função para o criação da comparação usando gradiente
combined_image = gradiente_horizontal('D:\Pos_IA\TCC\carroa_edit.jpg', 'D:\Pos_IA\TCC\carrob_edit.jpg')


# Salvar a imagem combinada
cv2.imwrite('combinacao_com_degrade.png', combined_image)

# Mostrar a imagem combinada
cv2.imshow('Combinacao com Degradê', combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()




In [ ]:
#fazendo um degrade da imagem por completo

# Carregar as imagens
# first_image_path = 'caminho_para_a_primeira_imagem.png'
# second_image_path = 'caminho_para_a_segunda_imagem.png'
first_image = cv2.imread(first_image_path)
second_image = cv2.imread(second_image_path)

# Redimensionar a segunda imagem para corresponder ao tamanho da primeira
second_image_resized = resize_image_to_match(first_image, second_image)

# Definir a transparência (alfa) para cada imagem
alpha_first_image = 0.40  # 85% de transparência para a primeira imagem
alpha_second_image = 0.60 # 60% de transparência para a segunda imagem

# Calcular o alfa inverso para cada imagem
alpha_first_image_inv = 1.0 - alpha_first_image
alpha_second_image_inv = 1.0 - alpha_second_image

# Combinar as imagens com as transparências especificadas
combined_image = cv2.addWeighted(first_image, alpha_first_image, second_image_resized, alpha_second_image_inv, 0)

# Salvar e mostrar a imagem combinada
cv2.imwrite('combinacao_transparencia.png', combined_image)
cv2.imshow('Combinacao com Transparência', combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Alinhando e comparando as 3 imagens (veiculo novo, veiculo batido e um gradiente entre os dois)

# Alinhar as imagens lado a lado para comparação
combined_image_triple = np.hstack((imageA, imageB, combined_image))

# Salvar a imagem combinada
cv2.imwrite('combinacao_com_degrade.png', combined_image_triple)

# Mostrar a imagem combinada
cv2.imshow('Combinacao com Degradê', combined_image_triple)
cv2.waitKey(0)
cv2.destroyAllWindows()

